In [1]:
import itertools
import numpy
import urllib,json
import pickle
import sys,trace
import multiprocessing

RRR = RealField(100);
eps = var('eps');

#stolen from https://ask.sagemath.org/question/8390/creating-an-array-of-variables/
class VariableGenerator(object):
     def __init__(self, prefix): 
         self.__prefix = prefix 
     @cached_method 
     def __getitem__(self, key): 
         return SR.var("%s%s"%(self.__prefix,key))

#object: what to dump; name: string without ".pickle"
def pickletofile(object,name):
    thefile = open(name+".pickle",'wb');
    pickle.dump(object,thefile);
    thefile.close();
    return

#filename: string without ".pickle"
def unpicklefile(filename):
    thefile = open(filename+".pickle",'rb');
    theobj = pickle.load(thefile);
    thefile.close();
    return theobj

def savewits():
    pickletofile(SC,"stoch_complexity");
    pickletofile(SW,"witnesses");
    return;

#NOTE: use dictionary NC for strings of length 2 through 10.
#expects sigma to be an array/list, like usual
def nfacomplexity(sigma):
    theurl = "http://math.hawaii.edu/~bjoern/complexity-api/?string="+''.join(str(ch) for ch in sigma);
    response = urllib.request.urlopen(theurl);
    data = json.loads(response.read());
    return int(data.get('complexity'))

#returns bit flip of a string (given as a tuple)
def flip(sigma):
    l = [];
    for ch in sigma:
        l.append(1-ch);
    return tuple(l)

#flips every string in an iterable, returns a Set
def flipset(strings):
    theset = [];
    for s in strings:
        theset.append(flip(s));
    return Set(theset)

#switches the order of a pair of matrices
def switchmats(matpair):
    return {0: matpair[1], 1: matpair[0]}

#utility function: convert everything in an iterable to a tuple.
#returns a tuple
def maketuple(iterable):
    newit = [];
    for i in iterable:
        newit.append(tuple(i));
    return tuple(newit);

#return True iff NFA complexity of sigma is maximal
def isNFArand(sigma):
    assert sigma in NC.keys();
    return (NC[sigma] == floor(len(sigma)/2)+1);

#return True iff sigma uses at least nlett different letters
def usesletters(sigma,nlett):
    return (len(set(sigma)) >= nlett);

#returns "0.string" in base m. string should be a list or tuple
def madic_expansion(string, m):
    value = 0;
    counter = 0;
    while counter < len(string):
        assert string[counter] < m; #should actually be an m-adic string
        value = value + string[counter]*1/(m^(counter+1));
        counter = counter + 1;
    return value;

#return image of string under a homomorphism homo
#homo should be a dictionary {letter:image}; string a list or tuple
def homo_image(homo,string):
    image = [];
    for ch in string:
        assert ch in homo.keys(); #string should be in the domain of the homomorphism
        image = image + list(homo[ch]);
    return tuple(image);

#expects start_states & accept_states to be lists
#transmatrices should be a dictionary. Its keys will be interpreted
#as letters of the input alphabet for the PFA
def accprob(transmatrices,sigma,start_states=None,accept_states=None):
    alph = list(transmatrices.keys());
    assert len(alph)>0;
    n = transmatrices[alph[0]].nrows();
    Id = identity_matrix(n);
    if start_states == None:
        v = Matrix(Id[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(Id[n-1]).transpose();
    else:
        f = Matrix(accept_states).transpose();
    return accprob_vector(transmatrices,sigma,v,f);
    #A = v*Id;
    #for ch in sigma:
    #    assert ch in alph;
    #    A = A*transmatrices[ch];
    #A = A*f;
    #return A[0][0];

#expects start_states & accept_states to be matrices already
def accprob_vector(transmatrices,sigma,start_states,accept_states):
    A = start_states*probmatrix(transmatrices,sigma)*accept_states;
    return A[0][0];

#expects a PFA given as a "witness" tuple consisting of (matrixdict, start_states, accept_states)
def accprob_witness(thePFA,sigma):
    return accprob_vector(thePFA[0],sigma,thePFA[1],thePFA[2]);
 
#return transition probability matrix for the string sigma
#transmatrices is a matrix dict giving the trans matrices for each letter
def probmatrix(transmatrices,sigma):
    alph = list(transmatrices.keys());
    A = identity_matrix(transmatrices[alph[0]].nrows());
    for ch in sigma:
        assert ch in alph;
        A = A*transmatrices[ch];
    return A;

#lengths can be either an integer or a list
#return a list of pairs (string, accprob) for all strings of len in lengths.
#transmatrices should be a dictionary, {letter:transmatrix}
def list_probs(transmatrices,start_states=None,accept_states=None,lengths=range(1,5)): 
    alph = list(transmatrices.keys());
    assert len(alph)>0;
    n = transmatrices[alph[0]].nrows();
    Id = identity_matrix(n);
    if start_states == None:
        v = Matrix(Id[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(Id[n-1]).transpose();
    else:
        f = Matrix(accept_states).transpose();
    #if lengths is just a single integer, make it into a list
    if isinstance(lengths,sage.rings.integer.Integer):
        lenlist = [lengths];
    else:
        lenlist = lengths;
    
    problist = []; #list of strings with their acceptance probabilities
    for i in lenlist:
        l = list(itertools.product(alph,repeat=i));
        for st in l:
            A = v*Id;
            for ch in st:
                assert ch in alph;
                A = A*transmatrices[ch];
            A = A*f;
            problist.append((st,A[0][0]));
        
    return problist;

#expects to be passed a "witness" tuple, as from SW
def list_probs_witness(witness,lengths=range(1,5)):
    return list_probs(witness[0],witness[1].list(),witness[2].list(),lengths);

#pass a list of binary strings to test, instead of a range
def list_probs_fromwords(transmatrices,start_states,accept_states,words=None):
    alph = list(transmatrices.keys());
    n = transmatrices[alph[0]].nrows();
    Id = identity_matrix(n);
    v = Matrix(start_states);
    f = Matrix(accept_states).transpose();
    if words == None:
        words = list(itertools.product(alph,repeat=11));
    
    problist = []; #list of strings with their acceptance probabilities
    for st in words:
        A = v*Id;
        for ch in st:
            assert ch in alph;
            A = A*transmatrices[ch];
        A = A*f;
        problist.append((st,A[0][0]));
        
    return problist;

#expects a list in exactly the format returned by list_probs. if length>0, only looks at strings of exactly that length
def highest_prob(problist,length=0):
    assert length >= 0;
    if length == 0:
        sublist = problist;
    elif length > 0:
        sublist = [l for l in problist if len(l[0]) == length];
    highprob = max([l[1] for l in sublist]);
    highstr = [st for st in sublist if st[1] == highprob];
    return highstr;
    
def sortproblist(problist):
    return sorted(problist,key=lambda x:x[1],reverse=True);
    
#check transition matrices for rows summing to 1
#P should be a dictionary of matrices
def checkmatrices(P):
    alph = list(transmatrices.keys());
    n = P[keys[0]].nrows();
    
    for i in keys:
        for j in range(n):
            s = sum(P[i][j]);
            if s != 1:
                if s > 1: relto1 = " > 1";
                elif s < 1: relto1 = " < 1";
                print("P"+str(i)+" row "+str(j+1)+" sum is "+str(RRR(s))+relto1);

In [2]:
#given list of row vectors and alphabet, returns all possible PFAs (matrix dicts) using those rows.
#vecs = list of vectors (which should be tuples!)
#if makerational=True, make all the matrices in the end over QQ. Don't bother otherwise
def bruteforcevecs(vecs,alph=range(2),permute=True,info=False,makerational=True):
    nstates = len(vecs[0]);
    mats = Tuples(vecs,nstates);
    #now convert every single element of mats to a tuple (immutable)
    tmats = maketuple(mats);
    if info: print(str(len(tmats))+" matrices");
    #tmats = pool of possible matrices to pull from. Now build all
    #possible dictionaries with alph as keys.
    numletters = len(alph);
    if permute:
        mats2 = list(Tuples(tmats,numletters));
    else:
        mats2 = list(itertools.combinations(tmats,numletters));
    if info: print(str(len(mats2))+" matrix tuples");
    Pnew = []; #will be list of matrix dicts
    for P in mats2:
        newdict = {}; #we'll add to this a matrix for each letter, in order of appearance in P
        if makerational:
            for i in range(len(alph)):
                newdict.update({alph[i]:Matrix(QQ,P[i])});
        else:
            for i in range(len(alph)):
                newdict.update({alph[i]:Matrix(P[i])});
        Pnew.append(newdict);
    return Pnew;

#return list of dictionaries of nstates x nstates matrices, over
#the alphabet alph (which should be a list or tuple).
#(1/step) will be the increment of transition probabilities
#if permute=False, don't include (a) all ways to order a
#tuple of matrices; (b) tuples where any two mats are the same.
#if info=True, print sizes of stuff along the way
#if prec>0, make the entries have precision prec. If prec=0, they're in QQ
def bruteforce(nstates,step,alph=range(2),permute=True,info=False,prec=0):
    assert(prec >= 0);
    linit = Compositions(nstates+step,length=nstates);
    if info: print(str(len(linit))+" partitions");
    #lrows = [ [0]*nstates ];
    lrows = []; #not including rows of all 0s anymore
    if prec > 0:
        RRp = RealField(prec);
    for t in linit:
        r = []; #do arithmetic on t by hand
        for i in t:
            if prec == 0:
                r.append(QQ(i-1)/step);
            else:
                r.append(RRp(i-1)/step);
        lrows.append(tuple(r));
    trows = tuple(lrows);
    if info: print(str(len(trows))+" rows");
    return bruteforcevecs(trows,alph,permute,info,(prec==0));

#do bruteforce() algo using only the given list of transition probabilities as entries
#if oneminus=True: if a probability p is present but not 1-p, then 1-p will automatically be added
#if useeps=True, the matrices will just be over SR. Automatically overrides makerational to False.
#if makerational=True, convert all given probs to rationals
def bruteforceprobs(nstates,probs,alph=range(2),permute=True,info=False,useeps=False,makerational=True,oneminus=True):
    for p in probs:
        if p<0 or p>1:
            probs.remove(p);
    if makerational and not useeps:
        pro_list = [];
        for p in probs:
            pro_list.append(QQ(p));
        if oneminus:
            for p in pro_list:
                if pro_list.count(1-p) == 0:
                    pro_list.append(QQ(1-p));
        pro = tuple(pro_list);
    else:
        pro_list = copy(probs);
        if oneminus:
            for p in pro_list:
                if pro_list.count(1-p) == 0:
                    pro_list.append(1-p);
        pro = tuple(pro_list);
    if info: print(str(len(pro))+" possible entries");
    lrowsall = Tuples(pro,nstates).list();
    lrows = [];
    for r in lrowsall:
        s = sum(r);
        if s == 1: lrows.append(r);
    if info: print(str(len(lrows))+" rows");
    trows = maketuple(lrows);
    if not useeps:
        return bruteforcevecs(trows,alph,permute,info,makerational);
    else:
        return bruteforcevecs(trows,alph,permute,info,False);

#expects a list of dictionaries of matrices as returned by bruteforce()
#strlen = length of strings to try, accept_states = vector (list)
def finduniques(matslist,strlen,start_states=None,accept_states=None):
    alph = list(matslist[0].keys());
    n = matslist[0][alph[0]].nrows();
    uniques = [];
    if start_states == None:
        start_states = list(identity_matrix(n)[0]);
    if accept_states == None:
        accept_states = list(identity_matrix(n)[n-1]);
    words = list(itertools.product(alph,repeat=strlen));
    for P in matslist:
        probs = list_probs_fromwords(P,start_states,accept_states,words);
        high = highest_prob(probs);
        if len(high) == 1:
            uniques.append([P,high[0]]);
    return uniques;

In [3]:
#updates PFA complexities and adds new witnesses to SW
#uniqlist should be in the format output by finduniques()
#ASSUMES ALL MATRICES IN uniqlist ARE OF THE SAME SIZE!!!!!
#specify the _same_ list of accept states for all of them.
#return number of new witnesses added
def addwits(uniqlist,start_states=None,accept_states=None):
    alph = list(uniqlist[0][0].keys());
    n = uniqlist[0][0][alph[0]].nrows();
    count = 0;
    if start_states == None:
        v = Matrix(identity_matrix(n)[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
    else:
        f = Matrix(accept_states).transpose();
    for u in uniqlist:
        s = u[1][0];
        count = count + addonewit(u[0],s,start_states,accept_states);
    return count;

#like addwits(), but only adds if there isn't a witness yet for
#each particular string
#uniqlist should be in the format output by finduniques()
#ASSUMES ALL MATRICES IN uniqlist ARE OF THE SAME SIZE!!!!!
#specify the _same_ list of accept states for all of them.
#return number of new witnesses added
def addwitsnew(uniqlist,start_states=None,accept_states=None):
    alph = list(uniqlist[0][0].keys());
    n = uniqlist[0][0][alph[0]].nrows();
    count = 0;
    for u in uniqlist:
        s = u[1][0];
        if s in SW.keys() and len(SW[s]) > 0 and s in SC.keys():
            continue;
        count = count + addonewit(u[0],s,start_states,accept_states);
    return count;

#adds thewit (a matrix dict) as a witness for sigma to SW
#also updates SC if appropriate
#only does it if this witnesses minimal complexity
#return 0 if nothing was added or updated;
#       1 if only a new witness;
#       2 if new witness and complexity updated.
def addonewit(thewit,sigma,start_states=None,accept_states=None):
    n = thewit[sigma[0]].nrows();
    ret = 0;
    if start_states == None:
        v = Matrix(identity_matrix(n)[0]);
    else:
        assert n == len(start_states);
        v = Matrix(start_states);
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
    else:
        assert n == len(accept_states);
        f = Matrix(accept_states).transpose();
    if not sigma in SC.keys() or SC[sigma] > n:
        SC[sigma] = n;
        ret = ret + 1;
    if SC[sigma] == n:
        if not sigma in SW.keys(): SW[sigma] = [];
        newwit = (thewit,v,f);
        if not newwit in SW[sigma]:
            SW[sigma].append(newwit);
            ret = ret + 1;
    return ret;

#------------------------------------------------------------------------------------------------------

#same as addonewit(), but start/accept_states should now already be a
#matrix in the correct form
def addonewit_vec(thewit,sigma,start_states,accept_states):
    n = thewit[0].nrows();
    ret = 0;
    if not SC.has_key(sigma) or SC[sigma] > n:
        SC[sigma] = n;
        ret = ret + 1;
    if SC[sigma] == n:
        try:
            ind = SW[sigma].index((thewit,start_states,accept_states));
        except ValueError:
            SW[sigma].append((thewit,start_states,accept_states));
            ret = ret + 1;
    return ret;
    

#among PFAs given by matdicts, returns the first found which
#has sigma as its highest-prob word among strings of len(sigma).
#assumes all matrices have the same size.
#if stop>0, find the first stop witnesses (if possible).
#if stop=0, find all witnesses, if possible.
def findwitness(matdicts,sigma,start_states=None,accept_states=None,stop=1):
    n = matdicts[0][sigma[0]].nrows();
    if start_states == None:
        v = Matrix(identity_matrix(n)[0]);
    else:
        v = Matrix(start_states);
    if accept_states == None:
        f = matrix(identity_matrix(n)[n-1]).transpose();
    else:
        f = matrix(accept_states).transpose();
    words = list(itertools.product(list(matdicts[0].keys()),repeat=len(sigma)));
    words.remove(sigma);
    
    found = 0;
    wits = [];
    for P in matdicts:
        sigprob = accprob_vector(P,sigma,v,f);
        if sigprob == 0:
            continue;
        iscandidate = True;
        for s in words:
            testprob = accprob_vector(P,s,v,f);
            if testprob >= sigprob:
                iscandidate = False;
                break;
        if iscandidate:
            if stop != 1:
                wits.append((P,v,f));
            elif stop == 1:
                return P; #the traditional behavior: don't put the single witness into a list
            found = found + 1;
            if stop > 0 and found >= stop:
                return wits;
    if len(wits) == 0:
        return None;
    else:
        return wits;

#expects a list in the format given by finduniques
#sigma should be a TUPLE, not an array! (?)
#returns the first pair of matrices in uniqlist witnessing the
#PFA complexity of sigma
def firstwitness(uniqlist,sigma):
    return [u[0] for u in uniqlist if u[1][0][0] == sigma][0]

#expects a dictionary like SC or NC
#returns Set of keys in compdict with value=complexity
#restricts to length strlen if strlen>0; o/w no restr on length
#if less=True, returns keys with value<=complexity
def lookup(compdict,complexity,strlen=0,less=False):
    if strlen == 0 and not less:
        return Set([k for k in compdict.keys() if compdict[k] == complexity]);
    elif strlen > 0 and not less:
        return Set([k for k in compdict.keys() if compdict[k] == complexity and len(k) == strlen]);
    elif strlen == 0 and less:
        return Set([k for k in compdict.keys() if compdict[k] <= complexity]);
    elif strlen > 0 and less:
        return Set([k for k in compdict.keys() if compdict[k] <= complexity and len(k) == strlen]);


#return largest denominator in a pair of rational matrices
def matdenom(matpair):
    themax = 1;
    for i in range(2):
        for r in matpair[i]: #rows
            for e in r: #element
                d = QQ(e).denominator();
                themax = lcm(d,themax);
    return themax;

#return first witness for PFA complexity of sigma in SW with
#matdenom <= denom
def firstwitdenom(sigma,denom):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    for mats in SW[sigma]:
        if matdenom(mats[0]) <= denom:
            return mats;
    return None;

#return list of witnesses for PFA complexity of sigma in SW
#with matdenom = denom
def witsdenom(sigma,denom):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    thelist = [];
    for mats in SW[sigma]:
        if matdenom(mats[0]) == denom:
            thelist.append(mats);
    if thelist == []:
        return None;
    return thelist;

#return lowest denominator out of all witnesses for sigma in SW
def mindenom(sigma):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    return min([matdenom(w[0]) for w in SW[sigma]]);

#return highest denominator out of all witnesses for sigma in SW
def maxdenom(sigma):
    if not SW.has_key(sigma) or len(SW[sigma]) == 0:
        return None;
    return max([matdenom(w[0]) for w in SW[sigma]]);

#determine if matpair represents an actual aut (as in Rabin)
def isactualaut(matpair):
    for i in range(2):
        for r in matpair[i]: #rows
            for e in r: #entries
                if e == 0:
                    return False;
    return True;

In [18]:
#substitute epses (tuple of epsilons, one tuple per matrix,
#one epsilon per row) into matpairs for the variable eps
def matsub(matpairs,epses):
    assert len(epses[0]) == len(epses[1]) == matpairs[0].nrows();
    newmat = [ [], [] ];
    for i in range(2):
        for e in range(len(epses[i])):
            newmat[i].append(matpairs[i][e].substitute(eps=epses[i][e]));
    return [Matrix(QQ,newmat[0]), Matrix(QQ,newmat[1])];

#just returns the digraph of the aut described by matpair
def autgraph(matpair):
    n = matpair[0].nrows();
    gr = DiGraph(n,loops=True,multiedges=True);
    for l in range(2): #letter seen
        for i in range(n): #transition from state i...
            for j in range(n): #to state j
                if matpair[l][i][j] > 0:
                    gr.add_edge(i,j,label=str(l)+" ("+str(matpair[l][i][j])+")");
    return gr;

#plots the specified aut from SW
#its=# of iterations for plotting algo
def autplotwit(sigma,ind,its=2):
    matpair = SW[sigma][ind][0];
    f = SW[sigma][ind][1];
    n = f.nrows();
    gr = autgraph(matpair);
    acc = [i for i in range(n) if f[i][0] == 1];
    notacc = Set(range(n)).difference(Set(acc)).list();
    return gr.plot(edge_labels=True,vertex_size=400,layout='spring',iterations=its,vertex_colors={'white': notacc,'yellow': acc});

#expects a matrix dict defining the automaton
def autplot(matdict,accept_states=None,its=2):
    n = matpair[matdict.keys()[0]].nrows();
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
        acc = [n-1];
    else:
        f = Matrix(accept_states).transpose();
        acc = [i for i in range(n) if accept_states[i] == 1];
    gr = autgraph(matpair);
    notacc = Set(range(n)).difference(Set(acc)).list();
    return gr.plot(edge_labels=True,vertex_size=400,layout='spring',iterations=its,vertex_colors={'white': notacc,'yellow': acc});

#expects a digraph (probably the output of autgraph())
def autgraphplot(autg,accept_states=None,its=2):
    n = autg.order();
    if accept_states == None:
        f = Matrix(identity_matrix(n)[n-1]).transpose();
        acc = [n-1];
    else:
        f = Matrix(accept_states).transpose();
        acc = [i for i in range(n) if accept_states[i] == 1];
    notacc = Set(range(n)).difference(Set(acc)).list();
    return autg.plot(edge_labels=True,vertex_size=400,layout='spring',iterations=its,vertex_colors={'white': notacc,'yellow': acc});

#given PFA (as matrix dictionary and init/acc states) and string sigma, 
#print a list of all accepting paths for sigma (as a sequence of 
#states) and the probability accumulated along each path
#accept_states must be a list/tuple
#for simplicity, for now only works for a single start state
def pathlist(matdict,sigma,accept_states=None):
    alph = list(matdict.keys());
    n = matdict[alph[0]].nrows(); # #states
    l = len(sigma)+1; #length of path
    #if start_states == None:
    #    v = Matrix(identity_matrix(n)[0]);
    #else:
    #    v = Matrix(start_states);
    if accept_states == None:
        accept_states = matrix(identity_matrix(n)[n-1]).transpose().list();
    #list of all possible sequences of states of the correct length
    stateseqs = [];
    for i in range(n):
        if accept_states[i] == 1: #if ith state is accepting
            for tup in Tuples(range(n),l-2):
                tupli = list(tup);
                tupli.insert(0,0); #begin with state 0 (before making any state transition)
                tupli.append(i); #end with state i, which we know is accepting
                stateseqs.append(tupli);
    #list of all such paths with their probs
    seqsnprobs = [];
    for aseq in stateseqs:
        pathprob = 1;
        for k in range(l-1):
            pathprob = pathprob*matdict[sigma[k]][aseq[k]][aseq[k+1]];
        seqsnprobs.append((aseq,pathprob));
    theoutput = "";
    for s in seqsnprobs:
        if s[1] > 0:
            addstr = "path "+str(s[0])+" has prob "+str(s[1])+'\n';
            #print(addstr,flush=True)
            theoutput = theoutput + addstr;
    return theoutput;

#expects theaut to be a "witness" triple. Still (for now) ignores the actual initial state distribution
def pathlist_witness(theaut,sigma):
    return pathlist(theaut[0],sigma,theaut[2].list());

In [5]:
#run immediately when restarting
SC = unpicklefile("stoch_complexity");
NC = unpicklefile("nfa_complexity");
SW = unpicklefile("witnesses");


S2 = {}; #set of binary strings of reasonable length
S2[0] = Set([]);
for i in range(2,15):
    S2[i] = Set(itertools.product([0,1],repeat=i));
    S2[0] = S2[0].union(S2[i]);
    
S3 = {}; #ternary strings that use all 3 letters
S3all = {}; #all ternary strings
S3[0] = Set([]);
S3all[0] = Set([]);
for i in range(2,11):
    initial = list(itertools.product([0,1,2],repeat=i));
    final = [];
    for s in initial:
        if usesletters(s,3):
            final.append(s);
    S3[i] = Set(final);
    S3[0] = S3[0].union(S3[i]);
    S3all[i] = Set(initial);
    S3all[0] = S3all[0].union(S3all[i]);

In [ ]:
#STATS (binary strings only)
print(len(SC))
for l in range(3,11):
    if l<9:
        top=4;
    else:
        top=5;
    for c in range(1,top):
        print("len "+str(l)+" complexity "+str(c)+": "+str(lookup(SC,c,l).cardinality())+" (NFA: "+str(lookup(NC,c,l).cardinality())+")")
    nu = lookup(SC,100,l,less=True).cardinality();
    if nu < 2^l:
        print("total classified "+str(nu)+"/"+str(2^l));
    witsum = sum([len(SW[k]) for k in SW.keys() if len(k)==l]);
    print("total witnesses "+str(witsum));
    print("");
print("# strings of each length with PFA=NFA complexity:");
for l in range(3,11):
    print(str(l)+": "+str(len([k for k in SC.keys() if len(k) == l and SC[k] == NC[k]])))
print("# strings of each length with PFA=NFA complexity + 1:")
for l in range(3,11):
    print(str(l)+": "+str(len([k for k in SC.keys() if len(k) == l and SC[k] == NC[k] + 1])))

In [6]:
test2x2 = bruteforce(2,2);
test2x3 = bruteforce(2,3);
test2x4 = bruteforce(2,4);
test2x5 = bruteforce(2,5);
test2x6 = bruteforce(2,6);
test2x7 = bruteforce(2,7);
test2x8 = bruteforce(2,8);
test3x2 = bruteforce(3,2);
half3x3 = bruteforce(3,3,permute=False);

In [33]:
three2x2 = bruteforce(2,2,range(3));
three2x3 = bruteforce(2,3,range(3));
three2x4 = bruteforce(2,4,range(3));
three2x5 = bruteforce(2,5,range(3));
three2x6 = bruteforce(2,6,range(3));
three2x8 = bruteforce(2,8,range(3));

In [6]:
somevecs = [ [1,0,0],[0,1,0],[0,0,1],[1/4,1/4,1/2],
           [1/4,1/2,1/4],[1/2,1/4,1/4]];
some3x4 = bruteforcevecs(maketuple(somevecs),permute=True);

somevecs2 = [ [1,0,0],[0,1,0],[0,0,1], [1/4,0,3/4],
             [1/4,3/4,0], [3/4,0,1/4], [3/4,1/4,0],
             [0,1/4,3/4], [0,3/4,1/4]];
some3x4_2 = bruteforcevecs(maketuple(somevecs2),permute=False);

In [7]:
print(len(test3x2))
print(len(half3x3))
print(len(some3x4))
print(len(some3x4_2))

46656
499500
46656
265356


In [7]:
#given a list of strings (as tuples/lists), a pool of matrix dicts, and start/accept states,
#try to find a witness for each string in the pool.
#will print any successes, and stop after trying maxattempts strings. If maxattemps=0, try all of them.
#won't attempt to find witnesses if one already exists.
def tryfindwitness(strings,pool,start_states=None,accept_states=None,maxattempts=50):
    c = 0;
    for s in strings:
        if (s in SW.keys() and len(SW[s]) > 0) or NC[s] == 1:
            continue;
        c = c + 1;
        if c < 0:
            continue;
        if c > maxattempts and maxattempts != 0:
            break;
        #sys.stdout.write("%d\r" % (c) );
        print(c,s);
        v = start_states;
        f = accept_states;
        wits = findwitness(pool,s,v,f);
        if wits != None:
            print(s,wits,addonewit(wits,s,v,f));
            print("");
            print(flip(s),switchmats(wits),addonewit(switchmats(wits),flip(s),v,f));
            print("");
        else:
            print("no witness found");
            print("");

In [20]:
savewits()

In [8]:
rand10 = [s for s in NC.keys() if len(s)==10 and isNFArand(s)];
print(len(rand10))
rand11 = [s for s in NC.keys() if len(s)==11 and isNFArand(s)];
print(len(rand11))
rand12 = [s for s in NC.keys() if len(s)>=12 and isNFArand(s)];
print(len(rand12))

344
449
390


In [9]:
pool2_1 = test2x2+test2x3+test2x4+test2x5+test2x6
print(len(pool2_1))
pool2_2 = test2x8+bruteforce(2,10)
print(len(pool2_2))

4659
21202


In [35]:
test2x20 = bruteforce(2,20)
print(len(test2x20))

194481


In [10]:
pool3_1 = test3x2+half3x3
print(len(pool3_1))

546156


In [ ]:
tryfindwitness(rand12,pool3_1,[1,0,0],[0,0,1],maxattempts=10)

In [ ]:
tryfindwitness(rand12,pool3_1,[1,0,0],[1,0,0],maxattempts=10)

In [53]:
savewits()

In [20]:
#https://gist.github.com/the-moog/94b09b49232731bd2a3cedd24501e23b
import threading
from IPython.display import display
import ipywidgets as widgets
import time

def get_ioloop():
    import IPython, zmq
    ipython = IPython.get_ipython()
    if ipython and hasattr(ipython, 'kernel'):
        return zmq.eventloop.ioloop.IOLoop.instance()


#The IOloop is shared
ioloop = get_ioloop()


class MyThread(threading.Thread):
    def __init__(self):
        super().__init__()
        
        self.progress = widgets.FloatProgress(value=0, min=0, max=10)
        
        self._quit = threading.Event()
        self.start()
    
    def run(self):
        i = 0
        while not self._quit.isSet():
            def update_progress(i=i):
                if self._quit.isSet():
                    return
                if self.progress.value == 10:
                    self.progress.value = 0
                self.progress.value = i

            time.sleep(2)

            ioloop.add_callback(update_progress)

            i += 1
            if i > 10:
                i = 0
        self.progress.value = 10
        print("Quit")
            
    def quit(self):
        self._quit.set()

def run_progress():
    thread = MyThread()
    display(thread.progress)
    return thread

In [21]:
t1 = run_progress()

FloatProgress(value=0.0, max=10.0)

In [27]:
t1.quit()

Quit


In [ ]:
proc = multiprocessing.Process(target=tryfindwitness,args=(rand10,pool2_1,[1,0],[0,1],maxattempts=10));
proc.start()

In [146]:
#really inefficient (for larger sets of strings)
c = 0;
for s in rand5:
    #if len(SW[s]) > 0:
    #    continue;
    c = c + 1;
    if c < 0:
        continue;
    if c > 50:
        break;
    sys.stdout.write("%d\r" % (c) );
    v = [1,0];
    f = [0,1];
    #f = list(reversed(f));
    wits = findwitness(test4state[::],s,v,f);
    #wits = findwitness(half3x3[269502::91],s,v,f);
    #wits = findwitness(test2x15[::13],s,v,f);
    if wits != None:
        print(s,wits,addonewit(wits,s,v,f));
        print("");
        print(flip(s),switchmats(wits),addonewit(switchmats(wits),flip(s),v,f));
        print("");

In [ ]:
[(k,NC[k]) for k in NC.keys() if isNFArand(k) and len(k) < 5]

In [163]:
strs = [k for k in NC.keys() if isNFArand(k) and len(k) < 6];

In [89]:
#test if s is a substring of t. Expects them as iterables!
def testsubstr(s,t):
    if len(s) >= len(t) and s != t: return False;
    is_substr = False;
    for ind in range(len(t)-len(s)+1):
        if s == t[ind:ind+len(s)]:
            is_substr = True;
    return is_substr;

In [ ]:
for s in strs:
    for t in strs:
        #if s == t or s == tuple(reversed(t)) or s == tuple(flip(t)): continue;
        #avoid strings of same length, and duplication
        if len(s) >= len(t): continue
        if testsubstr(s,t): continue
        if testsubstr(s,tuple(reversed(t))): continue
        if testsubstr(s,tuple(flip(t))): continue
        if testsubstr(s,tuple(flip(reversed(t)))): continue
        #I KNOW there has to be a better way to do this
        con = []
        con.append([s+t,"s+t"])
        con.append([t+s,"t+s"])
        con.append([s+tuple(reversed(t)),"s+rev(t)"])
        con.append([s+tuple(flip(t)),"s+fl(t)"])
        con.append([s+tuple(flip(reversed(t))),"s+flrev(t)"])
        con.append([t+tuple(reversed(s)),"t+rev(s)"])
        con.append([t+tuple(flip(s)),"t+fl(s)"])
        con.append([t+tuple(flip(reversed(s))),"t+flrev(s)"])
        testrand = []
        for i in range(len(con)):
            teststr = con[i][0]
            if not (teststr in NC.keys()):
                #print(teststr)
                NC[teststr] = nfacomplexity(teststr)
                pickletofile(NC,"nfa_complexity")
                sleep(1)
            testrand.append(isNFArand(teststr))
        value = False #disjunction of all truth values - want to see if they all fail
        for i in range(len(testrand)):
            value = value or i
        #continue if any of them succeeded
        #if value: continue
        print(s,t)
        for i in range(len(con)):
            print(con[i],NC[con[i][0]])
        print("")

In [ ]:
#fully distinct strings of length 5 with A_P=3:
rand5 = [k for k in S2[5] if k[0]==0 and SC[k]==3]; rand5

In [45]:
test2x25 = bruteforce(2,25);
print(len(test2x25))

456976


In [49]:
uniq2x20 = finduniques(test2x20,strlen=5,start_states=[1,0],accept_states=[0,1]);

In [50]:
len(uniq2x20)

170548

In [50]:
def tracegaps(string,autlist,start=(1,0),accept=(0,1)):
    gap = QQ(1);
#FIXME: replace 2 with size of alphabet
    numhigher = 2^len(string) - 1;
    c = 0;
    for aut in autlist:
        c = c+1;
        sys.stdout.write("%d\r" % (c) );
        thelen = ZZ(len(string));
        thelist = sortproblist(list_probs(aut,start_states=start,accept_states=accept,lengths=thelen));
        highstr = highest_prob(thelist);
        #get index within thelist of string
        thelist_juststr = [s[0] for s in thelist];
        highstr_juststr = [s[0] for s in highstr];
        theind = thelist_juststr.index(string);
        #if string is among those with highest probability, print that info but
        #don't update the gap or index, because it could be a rather indiscriminate automaton.
        if string in highstr_juststr:
            if len(highstr_juststr) == 1:
                print("c = "+str(c));
                print("WITNESS FOUND",highstr,aut);
                print("");
            elif 1 < len(highstr_juststr) < numhigher: #exclude case where lots of strings are accepted with same prob
                print("c = "+str(c));
                print(str(string)+" among highest-prob strings: "+str(highstr),aut);
                print("");
        #logic: if theind < numhigher, only then decide whether to update the gap
        #if theind < numhigher and the gap is bigger, print that gap without updating it
        else:
            acc = accprob(aut,start_states=start,accept_states=accept,sigma=string);
            if acc == 0: continue;
            thisgap = highstr[0][1] - acc;
            if theind <= numhigher or numhigher == 0:
                if thisgap <=0: #this code should never execute
                    print("c = "+str(c));
                    print("Gap of "+str(thisgap)+" with index "+str(theind)+", not updating",aut);
                    print("");
                elif thisgap < gap and theind <= numhigher:
                    print("c = "+str(c));
                    gap = thisgap;
                    numhigher = theind;
                    print("Gap of "+str(gap)+" and non-increased index "+str(theind),aut);
                    print(thelist[0:theind+2]);
                    print("");
                elif thisgap >= gap and theind < numhigher:
                    print("c = "+str(c));
                    gap = thisgap; #experimental: tolerate increasing the gap if we get closer to the top
                    numhigher = theind;
                    print("Larger gap of "+str(thisgap)+", but new index "+str(theind),aut);
                    print(thelist[0:theind+2]);
                    print("");
    return;

In [86]:
test2state = bruteforcevecs(( #(0.99,0.01),(0.01,0.99),
                             (0.54,0.46),(0.46,0.54),
                             (0.55,0.45),(0.45,0.55),
                             (0.49,0.51),(0.51,0.49),
                             (0.89,0.11),(0.11,0.89),(0.88,0.12),(0.12,0.88),(0.545,0.455),(0.455,0.545),
                             (0.44,0.56),(0.56,0.44),(0.5425,1-0.5425),(1-0.5425,0.5425)),
                           makerational=True);
len(test2state)

65536

In [8]:
test3state = bruteforceprobs(2, [0.49,0.11,0.44,0.455,0.4475,0.45,0.4575,0.455,0.4525,0.46]);
len(test3state)

160000

In [ ]:
tracegaps( (0,1,1,0,1), test3state[21500:], (1,0),(0,1))

In [ ]:
tracegaps( (0,0,1,0,0), test3state, (1,0),(1,0))

In [102]:
tracegaps( (0,1,1,0,1), test2x25, (1,0),(0,1))

Gap of 3906/1953125 and new index 2 {0: [24/25  1/25]
[ 9/25 16/25], 1: [22/25  3/25]
[    1     0]}
[((0, 0, 0, 0, 1), 8559/78125), ((1, 1, 0, 0, 1), 42093/390625), ((0, 1, 1, 0, 1), 210069/1953125), ((1, 0, 1, 0, 1), 42003/390625), ((1, 0, 0, 0, 1), 8397/78125), ((1, 1, 1, 0, 1), 209907/1953125), ((0, 1, 0, 0, 1), 41931/390625), ((0, 0, 1, 1, 1), 209409/1953125), ((0, 1, 0, 1, 1), 209319/1953125), ((1, 1, 1, 1, 1), 1046343/9765625), ((1, 0, 0, 1, 1), 41853/390625), ((1, 0, 1, 1, 1), 209247/1953125), ((0, 1, 1, 1, 1), 1046181/9765625), ((0, 0, 1, 0, 1), 41841/390625), ((1, 1, 0, 1, 1), 209157/1953125), ((0, 0, 0, 1, 1), 41691/390625), ((0, 0, 0, 1, 0), 8309/78125), ((1, 1, 0, 1, 0), 40843/390625), ((0, 0, 1, 0, 0), 8159/78125), ((0, 1, 1, 1, 0), 203819/1953125), ((1, 0, 1, 1, 0), 40753/390625), ((1, 0, 0, 1, 0), 8147/78125), ((1, 1, 1, 1, 0), 203657/1953125), ((0, 1, 0, 1, 0), 40681/390625), ((0, 0, 1, 1, 0), 40591/390625), ((0, 1, 0, 0, 0), 8069/78125), ((1, 1, 1, 0, 0), 40093/390625

In [ ]:
tracegaps( (0,0,1,0,0), test2x20, (1,0),(0,1))

In [145]:
test4state = bruteforceprobs(2, [0.01,0.49,0.499,0.48,0.2,0.44,0.455,0.4475,0.45,0.4575,0.455,0.4525,0.46]);
len(test4state)

456976

In [5]:
probs5 = [0.01,0.001];
new1 = 0.430;
while new1 <= 0.460:
    probs5.append(new1);
    new1 = new1 + 0.005;
new2 = 0.490;
while new2 < 0.500:
    probs5.append(new2);
    new2 = new2 + 0.001;
test5state = bruteforceprobs(2,probs5);
print(len(test5state))

2085136


In [33]:
tracegaps( (0,1,1,0,1), test5state[200000:], (1,0),(0,1) )

c = 71
Gap of 616623/19531250 and non-increased index 4 {0: [99/100  1/100]
[43/100 57/100], 1: [ 89/200 111/200]
[113/200  87/200]}
[((0, 0, 0, 0, 1), 43167261/78125000), ((1, 0, 0, 0, 1), 42269973/78125000), ((1, 1, 0, 0, 1), 41777229/78125000), ((0, 1, 0, 0, 1), 41584953/78125000), ((0, 1, 1, 0, 1), 40700769/78125000), ((1, 1, 1, 0, 1), 40659567/78125000)]

c = 109
Gap of 125185023/4000000000 and non-increased index 4 {0: [ 99/100   1/100]
[ 87/200 113/200], 1: [ 89/200 111/200]
[113/200  87/200]}
[((0, 0, 0, 0, 1), 11051184507/20000000000), ((1, 0, 0, 0, 1), 21655153677/40000000000), ((1, 1, 0, 0, 1), 1337703177/2500000000), ((0, 1, 0, 0, 1), 2663319453/5000000000), ((0, 1, 1, 0, 1), 81447339/156250000), ((1, 1, 1, 0, 1), 325462683/625000000)]

c = 147
Gap of 15510627/500000000 and non-increased index 4 {0: [99/100  1/100]
[ 11/25  14/25], 1: [ 89/200 111/200]
[113/200  87/200]}
[((0, 0, 0, 0, 1), 11051547/20000000), ((1, 0, 0, 0, 1), 21667857/40000000), ((1, 1, 0, 0, 1), 13385277/

In [52]:
probs6 = [];
new1 = 0.450;
while new1 < 0.460:
    probs6.append(new1);
    new1 = new1 + 0.002;
new2 = 0.490;
while new2 <= 0.500:
    probs6.append(new2);
    new2 = new2 + 0.0005;
test6state = bruteforceprobs(2,probs6,info=True);
print(len(test6state))

51 possible entries
51 rows
2601 matrices
6765201 matrix tuples
6765201


In [53]:
tracegaps( (0,1,1,0,1), test6state, (1,0),(0,1) )

c = 2
Gap of 11/10000 and non-increased index 22 {0: [137/250 113/250]
[  11/20    9/20], 1: [11/20  9/20]
[11/20  9/20]}
[((0, 0, 0, 1, 0), 4511/10000), ((0, 0, 1, 1, 0), 4511/10000), ((0, 1, 0, 1, 0), 4511/10000), ((0, 1, 1, 1, 0), 4511/10000), ((1, 0, 0, 1, 0), 4511/10000), ((1, 0, 1, 1, 0), 4511/10000), ((1, 1, 0, 1, 0), 4511/10000), ((1, 1, 1, 1, 0), 4511/10000), ((0, 1, 0, 0, 0), 1127744511/2500000000), ((1, 1, 0, 0, 0), 1127744511/2500000000), ((0, 0, 0, 0, 0), 7048403193613/15625000000000), ((1, 0, 0, 0, 0), 563872255489/1250000000000), ((0, 0, 1, 0, 0), 2255489/5000000), ((0, 1, 1, 0, 0), 2255489/5000000), ((1, 0, 1, 0, 0), 2255489/5000000), ((1, 1, 1, 0, 0), 2255489/5000000), ((0, 0, 0, 0, 1), 9/20), ((0, 0, 0, 1, 1), 9/20), ((0, 0, 1, 0, 1), 9/20), ((0, 0, 1, 1, 1), 9/20), ((0, 1, 0, 0, 1), 9/20), ((0, 1, 0, 1, 1), 9/20), ((0, 1, 1, 0, 1), 9/20), ((0, 1, 1, 1, 1), 9/20)]

c = 52
Gap of 1127254509/1250000000000 and non-increased index 22 {0: [  11/20    9/20]
[137/250 113/250

KeyboardInterrupt: 

In [67]:
lookup(NC,5,8).intersection(lookup(SC,2,8))

{(0, 0, 0, 0, 1, 1, 1, 1), (1, 1, 1, 1, 0, 1, 0, 1), (0, 0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 0, 1, 1, 1, 0), (1, 1, 1, 1, 0, 0, 0, 0), (1, 1, 1, 1, 0, 0, 0, 1)}

In [114]:
thisx = (0, 0, 0, 0, 1, 0, 1, 0)
mingap = 0.001
v = (1,0)
f = (1,0)
f = list(reversed(f))
thiswits = findwitness(test2x8,thisx,v,f,stop=0)
for w in thiswits:
    slist = sortproblist(list_probs_fromwits(w,lengths=Integer(len(thisx))))
    thisgap = N(slist[0][1] - slist[1][1])
    if thisgap > mingap:
        print(w,N(slist[0][1]),"gap",thisgap)
        print("")

({0: [1/8 7/8]
[5/8 3/8], 1: [  0   1]
[3/4 1/4]}, [1 0], [0]
[1]) 0.592529296875000 gap 0.00109863281250000



In [ ]:
SW[thisx]

In [92]:
savewits()

In [ ]:
[s for s in SC.keys() if SC[s] >= floor(len(s)/2)+1]

In [10]:
some3x5 = bruteforceprobs(3,(0,1/5),permute=False,info=True)

4 possible entries
9 rows
729 matrices
265356 matrix tuples


In [17]:
for s in NC.keys():
    if len(s) > 6: continue;
    if not isNFArand(s): continue;
    test = False;
    if isNFArand(s+(0,)) or isNFArand(s+(1,)): test = True;
    if test == False:
        print(s,NC[s])
        print(s+(0,),NC[s+(0,)])
        print(s+(1,),NC[s+(1,)])
        print("")

(0, 0, 1, 1, 1) 3
(0, 0, 1, 1, 1, 0) 3
(0, 0, 1, 1, 1, 1) 3

(1, 1, 0, 0, 0) 3
(1, 1, 0, 0, 0, 0) 3
(1, 1, 0, 0, 0, 1) 3



In [22]:
SW[(0,1,1,0)]

[(
{0: [     1/100     99/100]                            
[5001/10000 4999/10000], 1: [51/100 49/100]         [1]
[ 1/100 99/100]}                           , [1 0], [0]
),
 (
{0: [  0   1]                      
[1/2 1/2], 1: [1/2 1/2]         [1]
[  0   1]}             , [1 0], [0]
),
 (
{0: [5/8 3/8]                      
[  1   0], 1: [3/4 1/4]         [1]
[1/2 1/2]}             , [1 0], [0]
)]

In [25]:
for w in SW[(0,1,1,0)]:
    slist = sortproblist(list_probs_fromwits(w,lengths=4))
    thisgap = N(slist[0][1] - slist[1][1])
    print(w,N(slist[0][1]),"gap",thisgap)
    print("")

({0: [     1/100     99/100]
[5001/10000 4999/10000], 1: [51/100 49/100]
[ 1/100 99/100]}, [1 0], [1]
[0]) 0.491523250000000 gap 0.0576232624500000

({0: [  0   1]
[1/2 1/2], 1: [1/2 1/2]
[  0   1]}, [1 0], [1]
[0]) 0.500000000000000 gap 0.0625000000000000

({0: [5/8 3/8]
[  1   0], 1: [3/4 1/4]
[1/2 1/2]}, [1 0], [1]
[0]) 0.750976562500000 gap 0.00292968750000000



In [ ]:
for w in SW[(0,1,1,0)]:
    slist = sortproblist(list_probs_fromwits(w,lengths=5))
    print(w,slist)
    print("")

In [129]:
v = (1,0)
f = (0,1)
f=tuple(reversed(f))
uniq3 = finduniques(three2x2,7,v,f);
print(len(uniq3))

141


In [130]:
selectlist = [u for u in uniq3 if usesletters(u[1][0],3)];
print(len(selectlist))

6


In [132]:
addwitsnew(selectlist,v,f)

12

In [133]:
savewits()

In [124]:
print(len(S3[4].difference(Set(SC.keys()))))

6


In [8]:
sig = (0,1,1,0,1)
witsig = SW[(0,1,1,0,1)][0]
print(witsig)

({0: [1/2 1/2   0]
[  1   0   0]
[  1   0   0], 1: [1/2   0 1/2]
[  1   0   0]
[1/2 1/2   0]}, [1 0 0], [0]
[0]
[1])


In [154]:
pathlist(SW[(0,1,1,0,1)][0][0],(0,1,1,0,1),(0,0,1))

path [0, 0, 0, 0, 0, 2] has prob 1/32
path [0, 1, 0, 0, 0, 2] has prob 1/16
path [0, 0, 2, 0, 0, 2] has prob 1/32
path [0, 0, 2, 1, 0, 2] has prob 1/16
path [0, 0, 0, 2, 0, 2] has prob 1/16
path [0, 1, 0, 2, 0, 2] has prob 1/8


In [153]:
pathlist(SW[(0,1,1,0,1)][0][0],(0,0,0,0,1),(0,0,1))

path [0, 0, 0, 0, 0, 2] has prob 1/32
path [0, 1, 0, 0, 0, 2] has prob 1/16
path [0, 0, 1, 0, 0, 2] has prob 1/16
path [0, 0, 0, 1, 0, 2] has prob 1/16
path [0, 1, 0, 1, 0, 2] has prob 1/8


In [149]:
pathlist(SW[(0,1,1,0,1)][0][0],(1,1,0,1,0),(0,0,1))

In [151]:
sortproblist(list_probs_fromwits(SW[(0,1,1,0,1)][0],lengths=5))

[((0, 1, 1, 0, 1), 3/8),
 ((0, 0, 0, 0, 1), 11/32),
 ((0, 0, 0, 1, 1), 11/32),
 ((0, 0, 1, 0, 1), 11/32),
 ((0, 1, 0, 0, 1), 11/32),
 ((0, 1, 0, 1, 1), 11/32),
 ((1, 0, 0, 0, 1), 11/32),
 ((1, 0, 0, 1, 1), 11/32),
 ((1, 0, 1, 0, 1), 11/32),
 ((1, 1, 1, 0, 1), 11/32),
 ((1, 1, 0, 0, 1), 5/16),
 ((1, 1, 0, 1, 1), 5/16),
 ((0, 1, 1, 1, 1), 9/32),
 ((1, 1, 1, 1, 1), 9/32),
 ((0, 0, 1, 1, 1), 1/4),
 ((1, 0, 1, 1, 1), 1/4),
 ((0, 0, 0, 0, 0), 0),
 ((0, 0, 0, 1, 0), 0),
 ((0, 0, 1, 0, 0), 0),
 ((0, 0, 1, 1, 0), 0),
 ((0, 1, 0, 0, 0), 0),
 ((0, 1, 0, 1, 0), 0),
 ((0, 1, 1, 0, 0), 0),
 ((0, 1, 1, 1, 0), 0),
 ((1, 0, 0, 0, 0), 0),
 ((1, 0, 0, 1, 0), 0),
 ((1, 0, 1, 0, 0), 0),
 ((1, 0, 1, 1, 0), 0),
 ((1, 1, 0, 0, 0), 0),
 ((1, 1, 0, 1, 0), 0),
 ((1, 1, 1, 0, 0), 0),
 ((1, 1, 1, 1, 0), 0)]

In [6]:
a,b,c,d,e,f,g,h,i=var('a,b,c,d,e,f,g,h,i')

In [157]:
M = Matrix( [[ a,b,c,(a+b)/2,(a+b)/2,c],
             [d,e,f,d,e,f],
             [g,h,i,g,h,i]] )

In [167]:
Matrix(QQ, [[1/16,5/16,5/8, 1/4,1/4,1/2],
            [0,1,0,0,1,0],
            [0,0,1,0,0,1]]).rref()

[ 1  0  0  4 -1 -2]
[ 0  1  0  0  1  0]
[ 0  0  1  0  0  1]

In [175]:
Matrix([[a,b,c,a,c,b],
            [0,1,0,0,1,0],
            [0,0,1,0,0,1]]).rref()

[         1          0          0          1 -b/a + c/a  b/a - c/a]
[         0          1          0          0          1          0]
[         0          0          1          0          0          1]

In [ ]:
Matrix( [[1/16,5/16,5/8],[0,1,0],[0,0,1]]) * Matrix([[4,-1,-2],[0,1,0],[0,0,1]]) * Matrix( [[1/12,1/3,7/12],[0,1,0],[0,0,1]])

[ 1/48   1/3 31/48]
[    0     1     0]
[    0     0     1]

In [176]:
Matrix([0,0,1]).transpose()*Matrix([1,0,0])

[0 0 0]
[0 0 0]
[1 0 0]

In [104]:
#returns an m-adic PFA, a la Salomaa/Turakainen. It is a tuple: (matrices, initial, accepting) (as in SW)
#input: homo, a dict giving a homomorphism. Keys are generators of the language, values are their images
#    keys should be numbers, values should be tuples
def madic(homo):
    alph = list(homo.keys());
    m = max(alph)+1;
    themats = {}; #matrix dict to generate
    for letter in alph:
        phi = madic_expansion(homo[letter],m);
        malpha = m^(-len(homo[letter]));
        themats.update({letter: Matrix(QQ, [[malpha, 1-malpha-phi, phi],
                                           [0,1,0],
                                           [0,0,1]])});
    return (themats, Matrix([1,0,0]), Matrix([0,0,1]).transpose());

#given a PFA in "witness" form, return a new PFA whose transition matrices are multiplied by the matrices
#in mask. That is, matrices[letter] becomes matrices[letter]*mask[letter]. (Right multiplication by default,
#but set left=True if you want left multiplication.)
#Does NOT check the types of anything or the sizes of matrices
def multiplyPFA(thePFA,mask,left=False):
    newmats = {};
    for letter in thePFA[0].keys():
        assert letter in mask.keys();
        if left:
            newmats.update({letter:mask[letter]*thePFA[0][letter]});
        else:
            newmats.update({letter:thePFA[0][letter]*mask[letter]});
    return (newmats,thePFA[1],thePFA[2]);

In [142]:
testhom = {0: (0,), 1: (1,)};
testmad = madic(testhom)
print(testmad)

({0: [1/2 1/2   0]
[  0   1   0]
[  0   0   1], 1: [1/2   0 1/2]
[  0   1   0]
[  0   0   1]}, [1 0 0], [0]
[0]
[1])


In [150]:
col23switch = Matrix([[1,0,0],[0,0,1],[0,1,0]]);
row23switch = Matrix([[1,1,-1],[0,0,1],[0,1,0]]);
testmask = {0: col23switch, 1:col23switch};
newtestmad = multiplyPFA(testmad,testmask);
print(newtestmad)

({0: [1/2   0 1/2]
[  0   0   1]
[  0   1   0], 1: [1/2 1/2   0]
[  0   0   1]
[  0   1   0]}, [1 0 0], [0]
[0]
[1])


In [151]:
for entry in sortproblist(list_probs_witness(testmad,lengths=2)):
    print(entry,homo_image(testhom,entry[0]))

((1, 1), 3/4) (1, 1)
((1, 0), 1/2) (1, 0)
((0, 1), 1/4) (0, 1)
((0, 0), 0) (0, 0)


In [155]:
for entry in sortproblist(list_probs_witness(newtestmad,lengths=range(3))):
    print(entry,entry[1].n().str(base=2))

((1, 0), 3/4) 0.11000000000000000000000000000000000000000000000000000
((0,), 1/2) 0.10000000000000000000000000000000000000000000000000000
((1, 1), 1/2) 0.10000000000000000000000000000000000000000000000000000
((0, 0), 1/4) 0.010000000000000000000000000000000000000000000000000000
((), 0) 0.0000000000000000000000000000000000000000000000000000
((1,), 0) 0.0000000000000000000000000000000000000000000000000000
((0, 1), 0) 0.0000000000000000000000000000000000000000000000000000


In [163]:
accprob_witness(newtestmad,(0,0,0,0,0,0,1)).n().str(base=2)

'0.10101000000000000000000000000000000000000000000000000'

In [5]:
#suspected witness for 01^n0
test010 = ({0:Matrix(QQ, [[0,1],[1/2,1/2]]), 1:Matrix(QQ,[[1/2,1/2],[0,1]])}, Matrix([1,0]), Matrix([1,0]).transpose());

In [24]:
list010 = list_probs_witness(test010,lengths=range(15))

In [25]:
for l in range(15):
    print(highest_prob(list010,l))

[((), 1)]
[((1,), 1/2)]
[((0, 0), 1/2)]
[((0, 1, 0), 1/2)]
[((0, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0), 1/2)]
[((0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0), 1/2)]


In [25]:
print(pathlist_witness(test010,(0,1,0,1,0)))

path [0, 1, 1, 0, 1, 0] has prob 1/8
path [0, 1, 1, 1, 1, 0] has prob 1/4

